In [21]:
!pip install categorical

^C
Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement categorical (from versions: none)
ERROR: No matching distribution found for categorical

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [174]:
import torch
print(torch.__version__)
from math import log

2.1.2+cpu


In [233]:
D = 4
H = 8
R0 = 0.1
hidden_size = 100

In [234]:
from collections import OrderedDict

model = torch.nn.Sequential(
    OrderedDict([
        ("linear_1", torch.nn.Linear(D * H, hidden_size)),
        ("activation_1", torch.nn.ReLU()),
        ("linear_2", torch.nn.Linear(hidden_size, D + 2))
    ])
)

In [235]:
optimizer = torch.optim.SGD(
    model.parameters(),  # Передаём все тензоры, учавствующие в градиентном спуске
    lr=0.01,  # learning rate - величина шага градиентного спуска
)


In [236]:
def RIGHT_staaate(state):
    global H, D
    res = []
    for i in range(0, D):
        for j in range(0, H):
            res.append(float(0) + (state[i] == j))
    return res

In [237]:
def reward(x):
    global R0, D, H
    f1 = True
    f2 = True
    for d in range(D):
        if not (0.25 < abs(x[d] / (H - 1) - 0.5) <= 0.5): f1 = False
        if not (0.3 < abs(x[d] / (H - 1) - 0.5) < 0.4): f2 = False
    return (R0 + 0.5 * f1 + 2 * f2)


In [238]:
def loss_fn():
    global H, D
    state = [0] * D
    way = []
    while True:
        pred = model(torch.tensor(RIGHT_staaate(state)))
        p_Fs = pred[0:-1]
        F = pred[-1]
        p_B = 0
        for i in range(0, D):
            if state[i] == H - 1:
                p_Fs[i] = -float("inf")
            if state[i] > 0:
                p_B += 1
        m = torch.nn.Softmax(dim=0)
        p_Fs = m(p_Fs)
        ind = torch.distributions.categorical.Categorical(p_Fs).sample()
        p_B = max(p_B, 1)
        way.append([F, torch.log(p_Fs[ind]), 1 / p_B])
        if ind == D:
            break
        state[ind] += 1
    loss = 0
    for i in range(0, len(way) - 1):
        loss += (way[i][0] + way[i][1] - way[i+1][0] - log(way[i + 1][2])) ** 2
    loss += (way[-1][0] + way[-1][1] - log(reward(state)) - log(1)) ** 2
    return loss, state

In [239]:
def random_WAy():
    global H, D
    state = [0] * D
    while True:
        pred = model(torch.tensor(RIGHT_staaate(state)))
        p_Fs = pred[0:-1]
        for i in range(0, D):
            if state[i] == H - 1:
                p_Fs[i] = -float("inf")
        m = torch.nn.Softmax(dim=0)
        p_Fs = m(p_Fs)
        ind = torch.distributions.categorical.Categorical(p_Fs).sample()
        if ind == D:
            break
        state[ind] += 1
    return state

In [240]:
 random_WAy()

[2, 1, 0, 0]

In [241]:
from itertools import product

rewards = torch.zeros(*[H for i in range(D)])
coord_diap = [range(H) for _ in range(D)]

for coord in product(*coord_diap):
    rewards[tuple(coord)] = reward(torch.tensor(coord))

rewards /= rewards.sum()

def loss2(a, samples=1000):
    counter = torch.zeros(*[H for i in range(D)])
    for i in range(samples):
        counter[tuple(a[i])] += 1

    counter /= counter.sum()

    return (rewards - counter).abs().sum()


In [ ]:
a = []
for i in range(0, 100000):
    optimizer.zero_grad()
    x = loss_fn()
    loss = x[0]
    smpl = x[1]
    a.append(smpl)
    loss.backward()
    optimizer.step()
    if i % 100 == 0:
        print(i)
        print(loss2(a[-10000:], len(a[-10000:])))

0
tensor(1.9979)
100
tensor(1.9473)
200
tensor(1.9070)
300
tensor(1.8789)
400
tensor(1.8461)
500
tensor(1.8120)
600
tensor(1.7713)
700
tensor(1.7275)
800
tensor(1.6776)
900
tensor(1.6389)
1000
tensor(1.6077)
1100
tensor(1.5772)
1200
tensor(1.5545)
1300
tensor(1.5258)
1400
tensor(1.5011)
1500
tensor(1.4816)
1600
tensor(1.4552)
1700
tensor(1.4325)
1800
tensor(1.4063)
1900
tensor(1.3850)
2000
tensor(1.3643)
2100
tensor(1.3481)
2200
tensor(1.3295)
2300
tensor(1.3120)
2400
tensor(1.2933)
2500
tensor(1.2772)
2600
tensor(1.2609)
2700
tensor(1.2469)
2800
tensor(1.2290)
2900
tensor(1.2119)
3000
tensor(1.1962)
3100
tensor(1.1807)
3200
tensor(1.1644)
3300
tensor(1.1499)
3400
tensor(1.1369)
3500
tensor(1.1239)
3600
tensor(1.1095)
3700
tensor(1.0935)
3800
tensor(1.0765)
3900
tensor(1.0642)
4000
tensor(1.0500)
4100
tensor(1.0379)
4200
tensor(1.0227)
4300
tensor(1.0109)
4400
tensor(0.9999)
4500
tensor(0.9882)
4600
tensor(0.9776)
4700
tensor(0.9666)
4800
tensor(0.9576)
4900
tensor(0.9493)
5000
tensor(